In [18]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [352]:
class Lipschitz_Linear(nn.Module):
    def __init__(self, h, activation = None):
        super(Lipschitz_Linear, self).__init__()
        self.B = 2 ## Upper bound on product of weights norms
        self.lip_reg = 0.0005 ## learning rate on Lip regularisation !
        self.order = float('inf') ### order of L_j norm
        h = h
        self.activation = activation
        layers = []
        self.linear_layers = []
        self.Norm_constraints = torch.rand(len(h) - 1) * self.B
        for layer in range(1,len(h)):
            linear = nn.Linear(h[layer -1], h[layer])
            layers.append(linear)
            layers.append(nn.BatchNorm1d(h[layer], affine=True))
            self.linear_layers.append(linear)
            if activation is not None:
                layers.append(self.activation())
        self.univariate_nn = nn.Sequential(*layers)
        
    def forward(self,x):
        return self.univariate_nn(x)
    
    def compute_constraint_gradient(self):
        self.grads_norm_constraints = []
        prod = torch.prod(self.Norm_constraints)
        for i in range(len(self.Norm_constraints)):
            grad = torch.sum(self.linear_layers[i].weight.grad * (self.linear_layers[i].weight.data / (torch.linalg.matrix_norm(self.linear_layers[i].weight.data, ord = self.order)))) + self.lip_reg * (prod / self.Norm_constraints[i])  * torch.exp(7 * ((prod) - self.B))
            self.grads_norm_constraints.append(grad)

    def upper_lipschitz_bound(self):
        return torch.prod(self.Norm_constraints)

    def update_norm_constraints(self):
        for i in range(len(self.Norm_constraints)):
            self.Norm_constraints[i] -= 0.001 * self.grads_norm_constraints[i]
    
    def project_on_norm_constraints(self):
        for i in range(len(self.Norm_constraints)):
            self.linear_layers[i].weight.data *= self.Norm_constraints[i] / torch.linalg.matrix_norm(self.linear_layers[i].weight.data, ord = self.order)
    
    def train_enforce_constraints(self):
        self.compute_constraint_gradient()
        self.update_norm_constraints()
        self.project_on_norm_constraints()





In [252]:
class Lipschitz_GRU(nn.Module):
    def __init__(self, in_dim ,hidden, depth = 2, activation = nn.ReLU):
        super(Lipschitz_GRU, self).__init__()
        self.hidden = hidden
        self.depth = depth

        self.B = 2 ## Upper bound on product of weights norms
        self.lip_reg = 0.0005 ## learning rate on Lip regularisation !
        self.order = float('inf') ### order of L_j norm
        self.activation = activation
        self.Norm_constraints = torch.rand(self.depth) * self.B
        self.gru = nn.GRU(in_dim, self.hidden, num_layers = self.depth, batch_first=True)
        
    def forward(self, x):
        out, self.hidden_state = self.gru(x, self.hidden_state)
        return out
    
    def init_hidden(self, batch_size):
        self.hidden_state = torch.zeros(self.depth, batch_size, self.hidden)
    
    def compute_constraint_gradient(self):
        self.grads_norm_constraints = []
        prod = torch.prod(self.Norm_constraints)
        for i in range(len(self.Norm_constraints)):
            grad = torch.sum(self.linear_layers[i].weight.grad * (self.linear_layers[i].weight.data / (torch.linalg.matrix_norm(self.linear_layers[i].weight.data, ord = self.order)))) + self.lip_reg * (prod / self.Norm_constraints[i])  * torch.exp(7 * ((prod) - self.B))
            self.grads_norm_constraints.append(grad)

    def upper_lipschitz_bound(self):
        return torch.prod(self.Norm_constraints)

    def update_norm_constraints(self):
        for i in range(len(self.Norm_constraints)):
            self.Norm_constraints[i] -= 0.001 * self.grads_norm_constraints[i]
    
    def project_on_norm_constraints(self):
        for i in range(len(self.Norm_constraints)):
            self.linear_layers[i].weight.data *= self.Norm_constraints[i] / torch.linalg.matrix_norm(self.linear_layers[i].weight.data, ord = self.order)
    
    def train_enforce_constraints(self):
        self.compute_constraint_gradient()
        self.update_norm_constraints()
        self.project_on_norm_constraints()





In [259]:
class GRU(nn.Module):
    def __init__(self, in_dim ,hidden, depth = 2):
        super(GRU, self).__init__()
        self.hidden = hidden
        self.depth = depth
        self.gru = nn.GRU(in_dim, self.hidden, num_layers = self.depth, batch_first=True)
        
    def forward(self, x):
        out, self.hidden_state = self.gru(x, self.hidden_state)
        return out
    
    def init_hidden(self, batch_size):
        self.hidden_state = torch.zeros(self.depth, batch_size, self.hidden)

In [367]:
class KAN_RNN_Layer(nn.Module):
    def __init__(self, N_Agents, in_dim, hidden, depth):
        """ 
        in_dim:Dimension of Agent information, i.e cartesian coordinates R^2
        """
        super(KAN_RNN_Layer, self).__init__()
        self.N_Agents = N_Agents
        self.in_dim = in_dim
        self.hidden = hidden 
        self.Network_stack = []
        self.linear_Network_stack = []
        self.activation = nn.ReLU()
        self.num_forward_steps = 10
        for _ in range(self.N_Agents):
            Networks = []
            for _ in range(self.N_Agents):
                Networks.append(Lipschitz_GRU(in_dim = self.in_dim ,hidden = self.hidden, depth = depth)) ## Dimension of input x -> indim, depth number of stacked Gru Layers, hidden Numer of Neurons in the GRu Layers
            self.Network_stack.append(Networks)
            self.linear_Network_stack.append(Lipschitz_Linear([N_Agents * hidden, self.in_dim]))

    def time_step(self, x):
        outs = torch.zeros_like(x)
        for i in range(self.N_Agents): ## out
            output_list = []
            for j in range(self.N_Agents): ### in
                output_list.append(self.Network_stack[i][j](x[:,j,:].unsqueeze(1)))
            out = self.linear_Network_stack[i](self.activation(torch.cat(output_list, dim=1).reshape(-1, self.N_Agents * self.hidden)))
            outs[i] = out
        return outs

    def system_dynamics(self,u, x_prev):
        lam = 0.5
        return x_prev + lam * u

    def forward(self, x):
        """
        x: Inital States [Batch_size, N_Agents, in_dim] 

        return 
        control_trajectory: controller output [Num_timesteps ,Batch_size, N_Agents, in_dim]
        outs: State of Agents [Num_timesteps ,Batch_size, N_Agents, in_dim]  

        """ 
        outs = torch.zeros(self.num_forward_steps, *x.shape)
        control_trajectory = torch.zeros(self.num_forward_steps, *x.shape) ## Assume u of same shape as x!
        for i in range(self.num_forward_steps):
            outs[i] = x
            u = self.time_step(x)
            x = self.system_dynamics(u, x)
            control_trajectory[i] = u
        return outs, control_trajectory


    def init_hidden(self, batch_size):
        for lists in self.Network_stack:
            for gru in lists:
                gru.init_hidden(batch_size)

    def train_enforce_constraints(self):
        Lip_lin.train_enforce_constraints()
        self.fc2.train_enforce_constraints()

         


In [378]:
import numpy as np
from scipy.integrate import dblquad

def evaluate_integral(k, L):
    """
    Evaluate the integral for multiple k and L values.
    
    Parameters:
    k: List of k values [k1, k2, ...]
    L: List of L values [L1, L2, ...]

    Returns:
    Tensor of the square root of the evaluated integrals.
    """
    def integrand(*args):
        result = 1.0
        for i in range(len(k)):
            result *= np.cos(k[i] * args[i])**2
        return result

    # Define the integration limits for all dimensions
    limits = [(0, L) for L in L]

    # Compute the nested integral
    integral, _ = nquad(integrand, limits)

    return torch.tensor(np.sqrt(integral))


In [379]:
from itertools import product
class Ergodicity_Loss(nn.Module):
    def __init__(self, N_Agents, n_timesteps):
        super(Ergodicity_Loss, self).__init__()
        self.N_Agents = N_Agents
        self.n_timesteps = n_timesteps
        self.L = [3,3]

    def compute_normalization_constant(k):
        """ 
        h_k
        """
        return evaluate_integral(k, self.L)

    def fourier_basis(x, k):
        """ 
        x: State at time t_k (N_Agents, in_dim)
        Assume L = 2
        """
        result = torch.tensor(1.0)
        for i in range(len(k)):
            result *= torch.cos(k[i] * x[:, i])
        result *= self.compute_normalization_constant(k)
        return result
        

    def compute_fourier_coefficients_agents(x, k):
        """
        x: State of Agents [Num_timesteps ,Batch_size, N_Agents, in_dim] 
        """
        # For now i just put as calculaated t 1s
        transform = fourier_basis(x.view(-1, self.in_dim),k)
        result = transform.view(self.n_timesteps, -1 , self.N_Agents)
        c_k = result.sum(dim=0).sum(dim=1, keepdim=True)         
        return c_k

    def compute_fourier_coefficients_density(x):
        """
        x: State of Agents [Num_timesteps ,Batch_size, N_Agents, in_dim] 
        """
        # For now i just put as calculaated t 1s
        transform = fourier_basis(x.view(-1, self.in_dim),k)
        result = transform.view(self.n_timesteps, -1 , self.N_Agents)
        c_k = result.sum(dim=0).sum(dim=1, keepdim=True)         
        return c_k

    def forward(x):
        """
        x: State of Agents [Num_timesteps ,Batch_size, N_Agents, in_dim] 
        """
        loss = torch.zeros(1)
        k = list(range(n+1))
        for sets in product(k, repeat = 4):
            loss += (self.compute_fourier_coefficients(x,k) - self.compute_fourier_coefficients_density(k))

        
        

In [372]:
test = KAN_RNN_Layer(N_Agents = 4, in_dim = 2, hidden = 3, depth = 2)
test.init_hidden(batch_size = 4)

In [ ]:
x = torch.randn(4,4,2)
x,u = test(x)

tensor([[[-0.5733, -1.1735],
         [-0.2110,  1.3537],
         [-0.8912,  0.4967],
         [ 1.6755, -0.6769]],

        [[-1.0037, -0.9008],
         [ 1.1723,  1.1508],
         [ 0.8098,  0.8319],
         [-0.9783, -1.0819]],

        [[-0.7334, -0.4555],
         [ 1.3093, -0.9915],
         [ 0.6001,  1.6201],
         [-1.1761, -0.1730]],

        [[-0.2924,  0.0295],
         [-1.2782,  0.0803],
         [ 0.0702, -1.4653],
         [ 1.5005,  1.3555]]], grad_fn=<CopySlices>)
tensor([[[ 0.0798, -0.9756],
         [-0.8564,  1.3121],
         [-0.8203,  0.6041],
         [ 1.5969, -0.9406]],

        [[-0.9404, -0.9134],
         [ 1.1987,  1.1471],
         [ 0.7762,  0.8373],
         [-1.0345, -1.0710]],

        [[-0.5329,  1.6368],
         [ 1.3360, -0.9537],
         [ 0.4941, -0.0795],
         [-1.2972, -0.6036]],

        [[ 0.2098, -0.1779],
         [-1.4176, -0.1371],
         [-0.1664, -1.2360],
         [ 1.3742,  1.5510]]], grad_fn=<CopySlices>)
tensor([[[ 0